In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
from typing import Tuple

In [126]:
tree = ET.parse('../data/building.xml')
roots = tree.getroot()
print(roots.attrib)
cityObjectMember = roots[2]
print(cityObjectMember.tag)

{}
{http://www.opengis.net/citygml/2.0}cityObjectMember


In [137]:
# オブジェクトを宣言
cityObjects = {"id":"this is id","summary":"thi is summary"}
print(cityObjects)
print(cityObjects["id"])
if "id" in cityObjects:
    print("id is in cityObjects")

{'id': 'this is id', 'summary': 'thi is summary'}
this is id
id is in cityObjects


In [153]:
def getChildren(df:pd.DataFrame, node:any, text:str, posList:str,id:str,lod:int) -> Tuple[pd.DataFrame, str, str]:
    text += node.tag
    if len(node) > 0:
        for child in node:
            text += "\n"

            # 建物IDの取得
            # child.attrib オブジェクトが "id" キーを持っているか確認
            if id == "" and "{http://www.opengis.net/gml}id" in child.attrib:
                id = child.attrib["{http://www.opengis.net/gml}id"]
                print("id is:"+id)
            else:
                print("id is not in child.attrib")

            # lodの取得
            if "{http://www.opengis.net/citygml/building/2.0}lod0FootPrint" in child.tag:
                lod = 0
            elif "{http://www.opengis.net/citygml/building/2.0}lod1Solid" in child.tag:
                lod = 1
            elif "{http://www.opengis.net/citygml/building/2.0}lod2Solid" in child.tag:
                lod = 2
            else:
                lod = lod

            df,text,posList = getChildren(df,child, text,posList,id,lod)
            if child.tag == "{http://www.opengis.net/gml}posList":
                posList += "\n"
                posList += child.text
                # textの中身の数を調べる
                positionNum = int(child.text.count(" ")/3)
                print(positionNum)
                for i in range(int(positionNum)):
                    new_row = {'id':id,'lod':lod, 'lat':child.text.split(" ")[i*3], 'lon':child.text.split(" ")[i*3 + 1], 'height':child.text.split(" ")[i*3 +2]}
                    # df = df.append(new_row, ignore_index=True)
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                    print(new_row)
    return df, text, posList

In [154]:
text = ""
posList = ""
df = pd.DataFrame(columns=['id', 'lod', 'lat', 'lon', 'height'])
df,text,posList = getChildren(df,cityObjectMember, text,posList,id="",lod=0)
with open("../data/building.txt", "w") as f:
    f.write(text)
    f.close()

with open("../data/posList.txt", "w") as f:
    f.write(posList)
    f.close()

df.to_csv("../data/building.csv", index=False)

id is:bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
id is not in child.attrib
9
{'id': 'bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15', 'lod': 0, 'lat': '35.048090569743714', 'lon': '136.85000990360808', 'height': '0'}
{'id': 'bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15', 'lod': 0, 'lat': '35.048095332150716', 'lon': '136.85003137991248', 'height': '0'}
{'id': 'bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15', 'lod': 0, 'lat': '35.048117308330646', 'lon': '136.8500244044445', 'height': '0'}
{'id': 'bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15', 'lod': 0, 'lat': '35.04815522852685', 'lon': '136.85001236817075', 'height': '0'}
{'id': 'bldg_c2562e63-bb03-42e7-a4b5-a4d0badabb15', 'lod': 0, 'lat': '35.048156868202156', 'lon': '136.8500195223482', 'height': '0'}
{'id': 'bldg_c2562

In [63]:
def newGetChildren(df:pd.DataFrame, node:ET.Element, text:str, posList:str,id:str,lod:int) -> Tuple[pd.DataFrame, str, str]:
    text += node.tag
    if len(node) > 0:
        for child in node.findall("{http://www.opengis.net/citygml/2.0}cityObjectMember"):
            text += "\n"

            # 建物IDの取得
            # child.attrib オブジェクトが "id" キーを持っているか確認
            if id == "" and "{http://www.opengis.net/gml}id" in child.attrib:
                id = child.attrib["{http://www.opengis.net/gml}id"]
                print("id is:"+id)
            else:
                print("id is not in child.attrib")

            # lodの取得
            if "{http://www.opengis.net/citygml/building/2.0}lod0FootPrint" in child.tag:
                lod = 0
            elif "{http://www.opengis.net/citygml/building/2.0}lod1Solid" in child.tag:
                lod = 1
            elif "{http://www.opengis.net/citygml/building/2.0}lod2Solid" in child.tag:
                lod = 2
            else:
                lod = lod

            df,text,posList = getChildren(df,child, text,posList,id,lod)
            if child.tag == "{http://www.opengis.net/gml}posList":
                posList += "\n"
                posList += child.text
                # textの中身の数を調べる
                positionNum = int(child.text.count(" ")/3)
                print(positionNum)
                for i in range(int(positionNum)):
                    new_row = {'id':id,'lod':lod, 'lat':child.text.split(" ")[i*3], 'lon':child.text.split(" ")[i*3 + 1], 'height':child.text.split(" ")[i*3 +2]}
                    # df = df.append(new_row, ignore_index=True)
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                    print(new_row)
    return df, text, posList

In [ ]:
text = ""
posList = ""
df = pd.DataFrame(columns=['id', 'lod', 'lat', 'lon', 'height'])
df,text,posList = getChildren(df,cityObjectMember, text,posList,id="",lod=0)
with open("../data/building.txt", "w") as f:
    f.write(text)
    f.close()

with open("../data/posList.txt", "w") as f:
    f.write(posList)
    f.close()

df.to_csv("../data/building.csv", index=False)